# Juntando as Peças 🧩

## (Fontes de Dados 🚰 ➕ Manipulação 🪡 ➕ Análise de Dados 🕵🏽) <sup>Alto Desempenho 🚀 🧞‍♂️</sup>

### Funções Auxiliares

In [1]:
!pip install --upgrade pandas --quiet


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import os, json, sqlite3, pandas as pd

In [3]:
def build_path(subfolder = 'raw'):
    folderpath = os.path.join(os.getcwd(), os.pardir, 
                              'project', 'data', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
    return folderpath

#### csv

In [4]:
def get_estados_georreferenciamento(
        filename='estados_georreferenciamento.csv'):
    filepath = os.path.join(build_path(), filename)
    
    return pd.read_csv(filepath)

def get_municipios_georreferenciamento(
        filename='municipios_georreferenciamento.csv'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_csv(filepath)

def get_covid_infections_and_deaths(
        filename='ALL_HIST_PAINEL_COVID.csv'):
    filepath = os.path.join(build_path(), filename)
    
    return pd.read_csv(filepath, sep=';', parse_dates = ['data'],\
                       dtype={'codmun': 'Int64', \
                              'codRegiaoSaude': 'Int64', \
                              'populacaoTCU2019': 'Int64', \
                              'casosAcumulado': 'Int64', \
                              'Recuperadosnovos': 'Int64', \
                              'emAcompanhamentoNovos': 'Int64', \
                              'interior/metropolitana': 'Int64'}, \
                       encoding='utf-8')

#### json

In [5]:
#fail
def get_estados_codigos(filename='estados_codigos.json'):
    filepath = os.path.join(build_path(), filename)
    
    return pd.read_json(filepath)

#solution
def get_estados_codigos(filename='estados_codigos.json'):
    filepath = os.path.join(build_path(), filename)
    
    with open(filepath) as jsonfile:
        return pd.json_normalize(json.load(jsonfile))

#fail
def get_estados_caracteristicas(filename='estados_caracteristicas.json'):
    filepath = os.path.join(build_path(), filename)

    return pd.read_json(filepath)

#solution
def get_estados_caracteristicas(filename='estados_caracteristicas.json'):
    filepath = os.path.join(build_path(), filename)

    with open(filepath) as jsonfile:
        return pd.json_normalize(json.load(jsonfile), \
                                 record_path='characteristics', \
                                 record_prefix='characteristics_', \
                                 meta='state',\
                                 meta_prefix='state_')

#big fail
def get_estados_vacinacao(filename='estados_vacinacao.json'):
    filepath = os.path.join(build_path(), filename)

    return pd.read_json(filepath)

#solution
def get_estados_vacinacao(filename='estados_vacinacao.json'):
    filepath = os.path.join(build_path(), filename)

    with open(filepath) as jsonfile:
        df = pd.json_normalize(json.load(jsonfile)['Paciente_Estado']['buckets'], \
                               record_path=['Data_Aplicacao_Vacina', ['buckets']], \
                               meta=['key', 'doc_count'], \
                               record_prefix='vacinacao_', \
                               meta_prefix='estado_').convert_dtypes()
        df['vacinacao_key'] = pd.to_datetime(df['vacinacao_key'], unit='ms')
        return df

#big fail
def get_municipios_vacinacao(filename='municipios_vacinacao.json'):
    filepath = os.path.join(build_path(), filename)

    return pd.read_json(filepath)

#solution
def get_municipios_vacinacao(filename='municipios_vacinacao.json'):
    filepath = os.path.join(build_path(), filename)

    with open(filepath) as jsonfile:
        df = pd.json_normalize(json.load(jsonfile)['Paciente_Municipio']['buckets'], \
                               record_path=['Data_Aplicacao_Vacina', ['buckets']], \
                               meta=['key', 'doc_count'], \
                               record_prefix='vacinacao_', \
                               meta_prefix='municipio_',).convert_dtypes()
        df['vacinacao_key'] = pd.to_datetime(df['vacinacao_key'], unit='ms')
        return df

#### sqlite

In [6]:
def get_municipios_codigos(filename='municipios_codigos.db'):
    filepath = os.path.join(build_path(), filename)

    conn = sqlite3.connect(filepath)
    municipios_codigos = pd.read_sql_query('select * from ibge', conn, index_col='id')

    return municipios_codigos

### Descrição dos Dados 🎲🕵️‍♀️

#### Dados de Georreferenciamento

##### Unidades Federativas

In [7]:
estados_georreferenciamento = get_estados_georreferenciamento()

In [8]:
estados_georreferenciamento.head()

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


In [9]:
estados_georreferenciamento.dtypes

codigo_uf      int64
uf            object
nome          object
latitude     float64
longitude    float64
regiao        object
dtype: object

In [10]:
estados_georreferenciamento.describe()

,codigo_uf,latitude,longitude
count,27.000000,27.000000,27.000000
mean,29.111111,-12.381111,-48.410370
std,13.024631,8.476122,9.291479
min,11.000000,-30.170000,-70.550000
25%,19.000000,-18.645000,-52.990000
50%,27.000000,-10.570000,-48.260000
75%,38.000000,-6.205000,-41.025000
max,53.000000,1.990000,-36.590000


##### Municípios

In [11]:
municipios_georreferenciamento = get_municipios_georreferenciamento()

In [12]:
municipios_georreferenciamento.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [13]:
municipios_georreferenciamento.dtypes

codigo_ibge       int64
nome             object
latitude        float64
longitude       float64
capital           int64
codigo_uf         int64
siafi_id          int64
ddd               int64
fuso_horario     object
dtype: object

In [14]:
municipios_georreferenciamento.describe()

,codigo_ibge,latitude,longitude,capital,codigo_uf,siafi_id,ddd
count,5.570000e+03,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000
mean,3.253591e+06,-16.449101,-46.231012,0.004847,32.377738,4519.878276,57.099461
std,9.849103e+05,8.287274,6.408578,0.069461,9.833862,3050.156056,25.422750
min,1.100015e+06,-33.686600,-72.899700,0.000000,11.000000,1.000000,11.000000
25%,2.512126e+06,-22.843875,-50.878525,0.000000,25.000000,1595.500000,35.000000
50%,3.146280e+06,-18.094300,-46.523200,0.000000,31.000000,4382.000000,55.000000
75%,4.119190e+06,-8.496445,-41.410775,0.000000,41.000000,7180.500000,82.000000
max,5.300108e+06,4.603140,-32.410700,1.000000,53.000000,9997.000000,99.000000


#### Dados de Casos e Óbitos por Covid-19 no Brasil

In [15]:
covid_infections_and_deaths = get_covid_infections_and_deaths()

In [16]:
covid_infections_and_deaths.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,<NA>,<NA>,NaN,2020-08-01,31,210147125,2707877,45392,93563,1088,1865729,748585,<NA>
1,Brasil,NaN,NaN,76,<NA>,<NA>,NaN,2020-08-02,32,210147125,2733677,25800,94104,541,1883677,755896,<NA>
2,Brasil,NaN,NaN,76,<NA>,<NA>,NaN,2020-08-03,32,210147125,2750318,16641,94665,561,1912319,743334,<NA>
3,Brasil,NaN,NaN,76,<NA>,<NA>,NaN,2020-08-04,32,210147125,2801921,51603,95819,1154,1970767,735335,<NA>
4,Brasil,NaN,NaN,76,<NA>,<NA>,NaN,2020-08-05,32,210147125,2857597,55676,97240,1421,2020637,741180,<NA>


In [17]:
covid_infections_and_deaths.dtypes

regiao                            object
estado                            object
municipio                         object
coduf                              int64
codmun                             Int64
codRegiaoSaude                     Int64
nomeRegiaoSaude                   object
data                      datetime64[ns]
semanaEpi                          int64
populacaoTCU2019                   Int64
casosAcumulado                     Int64
casosNovos                         int64
obitosAcumulado                    int64
obitosNovos                        int64
Recuperadosnovos                   Int64
emAcompanhamentoNovos              Int64
interior/metropolitana             Int64
dtype: object

In [18]:
covid_infections_and_deaths.describe()

,coduf,codmun,codRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
count,7.271854e+06,7234754.0,7207580.0,7271854,7.271854e+06,7244680.0,7271854.0,7.271854e+06,7.271854e+06,7.271854e+06,1324.0,1324.0,7207580.0
mean,3.236020e+01,325258.01413,32403.123698,2022-01-02 10:06:07.585595392,2.673846e+01,114403.984855,12088.290272,1.561497e+01,2.640036e+02,2.913186e-01,21099156.385952,512267.851964,0.0693
min,1.100000e+01,110000.0,11001.0,2020-02-25 00:00:00,1.000000e+00,781.0,0.0,-3.368370e+05,0.000000e+00,-9.114000e+03,0.0,-6206.0,0.0
25%,2.500000e+01,251200.0,25010.0,2021-02-13 00:00:00,1.500000e+01,5474.0,286.0,0.000000e+00,5.000000e+00,0.000000e+00,7502146.25,121936.25,0.0
50%,3.100000e+01,314610.0,31059.0,2022-01-02 00:00:00,2.700000e+01,11695.0,884.0,0.000000e+00,1.500000e+01,0.000000e+00,21473002.0,395619.5,0.0
75%,4.100000e+01,411915.0,41015.0,2022-11-22 00:00:00,3.800000e+01,25765.0,2443.0,1.000000e+00,4.200000e+01,0.000000e+00,34126544.75,738947.25,0.0
max,7.600000e+01,530010.0,53001.0,2023-10-11 00:00:00,5.300000e+01,210147125.0,37849919.0,3.369590e+05,7.061420e+05,9.115000e+03,37104058.0,11232608.0,1.0
std,9.874132e+00,98535.03101,9836.341989,NaN,1.427105e+01,2969868.733786,363200.396715,7.216812e+02,7.759715e+03,1.668868e+01,13325444.310171,584092.153503,0.253963


##### Características das Unidades da Federação

In [19]:
estados_caracteristicas = get_estados_caracteristicas()

In [20]:
estados_caracteristicas.head()

,characteristics_label,characteristics_value,characteristics_measure,state_state
0,Governador,MARCOS JOSÉ ROCHA DOS SANTOS,,ro
1,Capital,Porto Velho,,ro
2,Gentílico,rondoniense ou rondoniano,,ro
3,Área Territorial,"237.754,172",km²,ro
4,População residente,1.581.016,pessoas,ro


In [21]:
estados_caracteristicas.dtypes

characteristics_label      object
characteristics_value      object
characteristics_measure    object
state_state                object
dtype: object

In [22]:
estados_caracteristicas.describe()

,characteristics_label,characteristics_value,characteristics_measure,state_state
count,324,324,324,324
unique,12,320,8,27
top,Governador,1.010,,ro
freq,27,2,108,12


#### Dados de Vacinação

##### Unidades Federativas

In [23]:
estados_vacinacao = get_estados_vacinacao()

In [24]:
estados_vacinacao.head()

,vacinacao_key_as_string,vacinacao_key,vacinacao_doc_count,estado_key,estado_doc_count
0,2023-10-11T00:00:00.000Z,2023-10-11,1996,SP,167920785
1,2023-10-10T00:00:00.000Z,2023-10-10,2723,SP,167920785
2,2023-10-09T00:00:00.000Z,2023-10-09,3186,SP,167920785
3,2023-10-08T00:00:00.000Z,2023-10-08,28,SP,167920785
4,2023-10-07T00:00:00.000Z,2023-10-07,19734,SP,167920785


In [25]:
estados_vacinacao.dtypes

vacinacao_key_as_string    string[python]
vacinacao_key              datetime64[ms]
vacinacao_doc_count                 Int64
estado_key                 string[python]
estado_doc_count                    Int64
dtype: object

In [26]:
estados_vacinacao.describe()

,vacinacao_key,vacinacao_doc_count,estado_doc_count
count,30764,30764.0,30764.0
mean,2021-04-17 22:37:25.845000,18841.48404,23492394.759069
min,1899-12-30 00:00:00,1.0,1665428.0
25%,2021-06-05 00:00:00,244.0,7395969.0
50%,2022-03-17 00:00:00,3438.0,11228172.0
75%,2022-12-27 00:00:00,16077.25,29897599.0
max,2023-10-11 00:00:00,1418737.0,167920785.0
std,NaN,51165.719939,33642764.602491


##### Municípios

In [27]:
municipios_vacinacao = get_municipios_vacinacao()

In [28]:
municipios_vacinacao.head()

,vacinacao_key_as_string,vacinacao_key,vacinacao_doc_count,municipio_key,municipio_doc_count
0,2023-10-11T00:00:00.000Z,2023-10-11,451,355030,46503474
1,2023-10-10T00:00:00.000Z,2023-10-10,711,355030,46503474
2,2023-10-09T00:00:00.000Z,2023-10-09,982,355030,46503474
3,2023-10-08T00:00:00.000Z,2023-10-08,18,355030,46503474
4,2023-10-07T00:00:00.000Z,2023-10-07,4278,355030,46503474


In [29]:
municipios_vacinacao.dtypes

vacinacao_key_as_string    string[python]
vacinacao_key              datetime64[ms]
vacinacao_doc_count                 Int64
municipio_key              string[python]
municipio_doc_count                 Int64
dtype: object

In [30]:
municipios_vacinacao.describe()

,vacinacao_key,vacinacao_doc_count,municipio_doc_count
count,62159,62159.0,62159.0
mean,2022-02-20 21:04:25.313000,3483.797841,3820674.17748
min,1900-08-21 00:00:00,1.0,1154510.0
25%,2021-08-01 00:00:00,81.0,1383084.0
50%,2022-04-17 00:00:00,983.0,1963638.0
75%,2023-01-06 00:00:00,3436.0,3138390.0
max,2023-10-11 00:00:00,316423.0,46503474.0
std,NaN,10329.912266,6623652.504356


#### Dados de Códigos do IBGE

##### Unidades Federativas

In [31]:
estados_codigos = get_estados_codigos()

In [32]:
estados_codigos.head()

,id,sigla,nome,regiao.id,regiao.sigla,regiao.nome
0,11,RO,Rondônia,1,N,Norte
1,12,AC,Acre,1,N,Norte
2,13,AM,Amazonas,1,N,Norte
3,14,RR,Roraima,1,N,Norte
4,15,PA,Pará,1,N,Norte


In [33]:
estados_codigos.dtypes

id               int64
sigla           object
nome            object
regiao.id        int64
regiao.sigla    object
regiao.nome     object
dtype: object

In [34]:
estados_codigos.describe()

,id,regiao.id
count,27.000000,27.000000
mean,29.111111,2.555556
std,13.024631,1.395965
min,11.000000,1.000000
25%,19.000000,1.500000
50%,27.000000,2.000000
75%,38.000000,3.500000
max,53.000000,5.000000


##### Municípios

In [35]:
municipios_codigos = get_municipios_codigos()

In [36]:
municipios_codigos.head()

,state,city
id,,
1100015,RO,Alta Floresta D''Oeste
1100379,RO,Alto Alegre dos Parecis
1100403,RO,Alto Paraíso
1100346,RO,Alvorada D''Oeste
1100023,RO,Ariquemes


In [37]:
municipios_codigos.dtypes

state    object
city     object
dtype: object

In [38]:
municipios_codigos.describe()

,state,city
count,5570,5570
unique,27,5298
top,MG,Bom Jesus
freq,853,5
